# Running Different Permutations of Models
- example below is using the car data set and the cros_val_score to test different Decision Tree models for different X-variables
- first section is doing it manually, second section I've automated with a for loop
- I've also automated looking at different scoring metrics

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import (KFold, StratifiedKFold, GroupKFold)
print(os.getcwd())

H:\jpmDesk\Desktop\DS Oct 2023\Python Project


In [29]:
df = pd.read_csv("ExData\car_data.csv")
df.set_index(["User ID"], inplace=True)
df = pd.concat([df, pd.get_dummies(df['Gender'])], axis=1)

https://scikit-learn.org/stable/modules/model_evaluation.html

In [30]:
clf = tree.DecisionTreeClassifier(random_state=42)
scores = cross_val_score(clf, df[['Age', 'AnnualSalary', 'Female']], df['Purchased'], cv=5)
scores

array([0.88 , 0.875, 0.905, 0.875, 0.87 ])

In [31]:
X = df[['Age', 'AnnualSalary', 'Female']]

In [32]:
cross_val_score(clf, X, df['Purchased'], cv=5, scoring='precision')

array([0.81111111, 0.83950617, 0.8961039 , 0.84146341, 0.86666667])

In [33]:
cross_val_score(clf, X, df['Purchased'], cv=5, scoring='recall')

array([0.9125    , 0.85      , 0.8625    , 0.85185185, 0.80246914])

In [34]:
cross_val_score(clf, X, df['Purchased'], cv=5, scoring='f1')

array([0.85882353, 0.8447205 , 0.87898089, 0.84662577, 0.83333333])

In [35]:
cross_val_score(clf, X, df['Purchased'], cv=5, scoring='roc_auc')

array([0.88541667, 0.87083333, 0.89791667, 0.87130408, 0.85755784])

In [36]:
clf.fit(X, df['Purchased'])

DecisionTreeClassifier(random_state=42)

# Permutate Models

In [56]:
df = pd.read_csv("ExData\car_data.csv", index_col=['User ID'])
df = pd.concat([df, pd.get_dummies(df['Gender'])], axis=1)

In [82]:
clf = tree.DecisionTreeClassifier(random_state=42)
clf2 = clf = LogisticRegression(random_state=0)
X1_cols = ['Age', 'AnnualSalary', 'Female'] #all cols
X2_cols = ['Age', 'AnnualSalary']
X3_cols = ['Age']
X4_cols = ['AnnualSalary']
allXs = [X1_cols, X2_cols, X3_cols, X4_cols]
typesModels = [clf, clf2]

y = df['Purchased']
metrics = ['accuracy','recall','precision','f1','roc_auc']

In [83]:
X_train, X_test, y_train, y_test = train_test_split(df[X1_cols], y, test_size=0.2, random_state=0)
        #800 rows of testing, 200 of training

In [96]:
results = {}
for metric in metrics:
    models = []
    for cols in allXs:   
        scores = cross_val_score(clf, X_train[cols], y_train, cv=5, scoring=metric)
                        #only using the 800 training rows, but does more splits 
                        #run 5 tests --> each test might be doing 100 test and 700 train
        models.append({'Scores':scores,'Mean':scores.mean(),'STD':scores.std(),'Min':scores.min(),'Cols':cols})
    resultsTable = pd.DataFrame(models)
    results[metric] = resultsTable

In [85]:
pd.concat(results)

Scores      Mean  \
accuracy  0         [0.80625, 0.91875, 0.88125, 0.85625, 0.85]  0.862500   
          1         [0.80625, 0.90625, 0.85625, 0.85, 0.84375]  0.852500   
          2              [0.8, 0.85, 0.76875, 0.83125, 0.8375]  0.817500   
          3       [0.70625, 0.71875, 0.70625, 0.73125, 0.7125]  0.715000   
recall    0  [0.71875, 0.859375, 0.8153846153846154, 0.8461...  0.817163   
          1  [0.765625, 0.859375, 0.7846153846153846, 0.784...  0.801923   
          2  [0.671875, 0.71875, 0.5846153846153846, 0.7230...  0.693510   
          3  [0.546875, 0.53125, 0.5076923076923077, 0.5230...  0.544856   
precision 0  [0.7796610169491526, 0.9322033898305084, 0.883...  0.840225   
          1  [0.7538461538461538, 0.9016393442622951, 0.85,...  0.828916   
          2  [0.7962962962962963, 0.8846153846153846, 0.791...  0.826307   
          3  [0.660377358490566, 0.6938775510204082, 0.6875...  0.687325   
f1        0  [0.7479674796747967, 0.8943089430894309, 0.848...  0.827648   
          1  [0.7596899224806202, 0.88, 0.816, 0.8095238095...  0.814875   
          2  [0.728813559322034, 0.7931034482758621, 0.6725...  0.752999   
          3  [0.5982905982905983, 0.6017699115044248, 0.584...  0.606333   
roc_auc   0  [0.7916666666666667, 0.9081217447916666, 0.870...  0.856087   
          1  [0.7994791666666667, 0.8984375, 0.842672064777...  0.843033   
          2  [0.8541666666666666, 0.8712565104166667, 0.761...  0.845927   
          3  [0.724609375, 0.74169921875, 0.704777327935222...  0.727529   

                  STD       Min                         Cols  
accuracy  0  0.037081  0.806250  [Age, AnnualSalary, Female]  
          1  0.032016  0.806250          [Age, AnnualSalary]  
          2  0.029422  0.768750                        [Age]  
          3  0.009354  0.706250               [AnnualSalary]  
recall    0  0.051286  0.718750  [Age, AnnualSalary, Female]  
          1  0.032859  0.765625          [Age, AnnualSalary]  
          2  0.062564  0.584615                        [Age]  
          3  0.037467  0.507692               [AnnualSalary]  
precision 0  0.058021  0.779661  [Age, AnnualSalary, Female]  
          1  0.049173  0.753846          [Age, AnnualSalary]  
          2  0.033802  0.791667                        [Age]  
          3  0.029835  0.655738               [AnnualSalary]  
f1        0  0.047440  0.747967  [Age, AnnualSalary, Female]  
          1  0.038339  0.759690          [Age, AnnualSalary]  
          2  0.046657  0.672566                        [Age]  
          3  0.016958  0.584071               [AnnualSalary]  
roc_auc   0  0.037631  0.791667  [Age, AnnualSalary, Female]  
          1  0.031733  0.799479          [Age, AnnualSalary]  
          2  0.042847  0.761943                        [Age]  
          3  0.013276  0.704777               [AnnualSalary]

In [93]:
#Final Model
finalX = X_train[['Age', 'AnnualSalary']]
clf.fit(finalX, y_train)

DecisionTreeClassifier(random_state=42)

In [94]:
clf.score(finalX, y_train)

0.9925

In [95]:
clf.score(X_test[['Age', 'AnnualSalary']],y_test)

0.905

In [86]:
from sklearn.model_selection import cross_validate

In [88]:
scoring = ['accuracy','recall','precision', 'f1','roc_auc']
scores = cross_validate(clf, X_train[X1_cols], y_train, scoring=scoring)
scores

{'fit_time': array([0.00400209, 0.004498  , 0.00400019, 0.00600338, 0.00400543]),
 'score_time': array([0.01351666, 0.01251268, 0.0150032 , 0.01399398, 0.01199627]),
 'test_accuracy': array([0.80625, 0.91875, 0.88125, 0.85625, 0.85   ]),
 'test_recall': array([0.71875   , 0.859375  , 0.81538462, 0.84615385, 0.84615385]),
 'test_precision': array([0.77966102, 0.93220339, 0.88333333, 0.80882353, 0.79710145]),
 'test_f1': array([0.74796748, 0.89430894, 0.848     , 0.82706767, 0.82089552]),
 'test_roc_auc': array([0.79166667, 0.90812174, 0.8708502 , 0.85384615, 0.85595142])}